In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
sys.path.append('../..')

In [8]:
import pandas as pd
import deepmatcher as dm
from utils.explainer import Explainer
from utils.patternDiscovery import mineRules

In [19]:
pos_nn = pd.read_csv('experiment_results/testposNN_RNN.csv')
negatives = pd.read_csv('../../Structured/Amazon-Google/negatives.csv')
association_rules = mineRules(pos_nn,negatives,'title',min_confidence=0.8,min_support=0.1)
association_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_isleft,consequents_isright
3,(L_microsoft),(R_microsoft),0.243590,0.235043,0.235043,0.964912,4.105263,0.177789,21.801282,True,True
18,(L_adobe),(R_adobe),0.128205,0.175214,0.106838,0.833333,4.756098,0.084374,4.948718,True,True
23,(L_office),(R_microsoft),0.111111,0.235043,0.102564,0.923077,3.927273,0.076448,9.944444,True,True
25,"(L_office, L_microsoft)",(R_microsoft),0.106838,0.235043,0.102564,0.960000,4.084364,0.077453,19.123932,True,True


In [10]:
association_rules_onprice = mineRules(pos_nn,negatives,'price',min_confidence=0.5,min_support=0.05)
association_rules_onprice.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_isleft,consequents_isright


In [22]:
len(pos_nn)

234

In [33]:
train = pd.read_csv('../../Structured/Amazon-Google/merged_train.csv')
price_distribution = dict(train.groupby(by=['ltable_price','rtable_price']).size())

## Test patterns

In [20]:
rnn_model = dm.MatchingModel(attr_summarizer='rnn')
rnn_model.load_state('../../models/amazongoogle_rnn.pth')
explainer = Explainer(rnn_model,10)

In [21]:
%%capture
explanations = explainer.generateExplanations(pos_nn,negatives,'price',0.03,
                                              '../../Structured/Amazon-Google/test_positives.csv',1)

In [22]:
for df,lval,rval in explanations:
    print((lval,rval))
    print(len(df))

('495.0', '49.99')
141
('29.99', '61.8')
104
('99.99', '85.07')
109
('nan', '21.41')
151
('499.99', '219.99')
93
('89.99', '621.99')
171


In [ ]:
p1 =['microsoft windows','microsoft windows']
explainer.testRobustness(testset_path='../../Structured/Amazon-Google/test_positives.csv',
                         true_label=1,attribute='title',substitute_values=p1,
                         strategy='concat')

In [12]:
import numpy as np
p2 = ['239.99','109.99']
explainer.testRobustness(testset_path='../../Structured/Amazon-Google/positives.csv',
                         true_label=1,attribute='price',substitute_values=p2,
                         strategy='replace')

===>  PREDICT Epoch 18
Finished Epoch 18 || Run Time:    0.6 | Load Time:    1.0 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00

===>  PREDICT Epoch 18
Finished Epoch 18 || Run Time:    0.6 | Load Time:    1.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(990, 1009)

## Inspect NN on critical attributes for negatives

In [ ]:
negNN = pd.read_csv('experiment_results/testnegNN_RNN.csv')
trainPos = pd.read_csv('../../Structured/Amazon-Google/train_positives.csv')
transactions = preProcessNeighbors(negNN,trainPos,'price')